In [1]:
using PauliPropagation
using Random
using Optim
using Plots
Random.seed!(43)
using ReverseDiff: GradientTape, gradient!, compile, gradient
using LinearAlgebra
using StatsBase 
using GLM
using DataFrames
using CSV

# CDR for quantum dynamics
- TFIM Hamiltonian (constants not site-dependent) evolving with TDSE
- discretize the evolution steps and use first-order Trotter decomposition 

### Idea:
- DONE: Run "exact" evolution for small trotterized circuit
- DONE: Create near-Clifford circuits:replace some gates in Trotterized gate by close Cliffords - no VQA here so this is much simpler, find closest Clifford to the RZZ and to the RX (only have 2 angles in total), replace a portion randomly (no MCMC), keep the parameter of N = non-Cliffords const. st. system remains cl. scalable.
- DONE: create MWE of CDR within quantum dynamics
- which gates ones we replace has influence on the accuracy of the expectation value (principle of causal light cone as seen in vnCDR paper Piotr, is this automatically respected within Heisenberg picture? (backprop observable)), so we can aim to replace only gates that contribute to an expectation value (works only for single-qubit / local observables)
- another idea is to try the perturbation approach (all non-Cliffords) 

In [2]:
struct trotter_ansatz
    target_circuit::Vector{Gate}
    target_circuit_layer::Vector{Gate}
    topology::Vector{Tuple{Int64, Int64}}
    nqubits::Integer
    steps::Integer #layers
    time::Integer
    J::Float64
    h::Float64
    sigma_J::Float64
    sigma_h::Float64
    sigma_J_indices::Vector{Int64}
    sigma_h_indices::Vector{Int64}
    sigma_J_indices_layer::Vector{Int64}
    sigma_h_indices_layer::Vector{Int64}
end

In [3]:
function trotter_setup(nqubits::Integer, steps::Integer, time::Float64, J::Float64, h::Float64;topology = nothing)
    if isnothing(topology)
        topology = bricklayertopology(nqubits)
    end
    target_circuit = tfitrottercircuit(nqubits,steps,topology=topology) #start with RZZ layer
    target_circuit_layer = tfitrottercircuit(nqubits,1,topology=topology) #start with RZZ layer
    sigma_J = -2*T*J/steps
    sigma_h = 2*T*h/steps 

    sigma_J_indices = getparameterindices(target_circuit, PauliRotation, [:Z,:Z]) 
    sigma_h_indices = getparameterindices(target_circuit, PauliRotation, [:X])
    
    sigma_J_indices_layer = getparameterindices(target_circuit_layer, PauliRotation, [:Z,:Z])
    sigma_h_indices_layer = getparameterindices(target_circuit_layer, PauliRotation, [:X])
    
    return trotter_ansatz(target_circuit,target_circuit_layer, topology, nqubits, steps, time, J, h,sigma_J, sigma_h,sigma_J_indices, sigma_h_indices, sigma_J_indices_layer, sigma_h_indices_layer)
end

trotter_setup (generic function with 1 method)

In [4]:
function constrain_params(ansatz; layer = false)

    if layer == false
        nparams = countparameters(ansatz.target_circuit)
        thetas = zeros(nparams)
        thetas[ansatz.sigma_h_indices] .= ansatz.sigma_h
        thetas[ansatz.sigma_J_indices] .= ansatz.sigma_J

    else
        nparams = countparameters(ansatz.target_circuit_layer)
        thetas = zeros(nparams)
        thetas[ansatz.sigma_h_indices_layer] .= ansatz.sigma_h
        thetas[ansatz.sigma_J_indices_layer] .= ansatz.sigma_J

    end
    
    return thetas
end

constrain_params (generic function with 1 method)

In [5]:
function obs_magnetization(ansatz)
    magnetization = PauliSum(ansatz.nqubits)
    for i in 1:nq
        add!(magnetization,:Z,i)
    end
    magnetization = magnetization/nq
    return magnetization
end

obs_magnetization (generic function with 1 method)

In [6]:
function training_set_generation(ansatz::trotter_ansatz; num_samples::Int = 10, non_cliffs::Int = 30)
    nparams = countparameters(ansatz.target_circuit)
    cliffs = nparams - non_cliffs

    ratio = length(ansatz.sigma_J_indices)/(length(ansatz.sigma_h_indices) + length(ansatz.sigma_J_indices))
    num_h = Int(round((1-ratio)*cliffs))
    num_J = Int(round(ratio*cliffs))
    training_thetas_list = Vector{Vector{Float64}}()
    thetas = constrain_params(ansatz)
    for _ in 1:num_samples
        training_thetas = deepcopy(thetas)
        shuffled_sigma_h_indices =  Random.shuffle!(ansatz.sigma_h_indices)
        shuffled_sigma_J_indices = Random.shuffle!(ansatz.sigma_J_indices)
        selected_indices_h = shuffled_sigma_h_indices[1:num_h]
        selected_indices_J = shuffled_sigma_J_indices[1:num_J];   
        k_h = round(ansatz.sigma_h/(π/4))
        k_J = round(ansatz.sigma_J/(π/4))

        for i in selected_indices_h
            training_thetas[i] = k_h*π/4
        end
        for i in selected_indices_J
            training_thetas[i] = k_J*π/4
        end
        push!(training_thetas_list, training_thetas)
    end
    return training_thetas_list
end

training_set_generation (generic function with 1 method)

In [7]:
function exact_trotter_time_evolution(ansatz; special_theta = false,noisy_circuit = false,record = false)

    layer = record
    if special_theta == false
        thetas = constrain_params(ansatz,layer=layer)
    else
        thetas = special_theta
    end
    obs = obs_magnetization(ansatz)

    if record == false
        if noisy_circuit == false
            noisy_circuit = copy(ansatz.target_circuit)
        end
        psum = propagate!(noisy_circuit,obs,thetas)
    
        return overlapwithzero(psum)

    else 
        expvals_trotter = Float64[]
        if noisy_circuit == false
            noisy_circuit = copy(ansatz.target_circuit_layer)
        end
        
        for i in 1:ansatz.steps
            psum = propagate!(noisy_circuit,obs, thetas)
            push!(expvals_trotter, overlapwithzero(psum))
        end

        return expvals_trotter
        
    end
    

end

exact_trotter_time_evolution (generic function with 1 method)

In [8]:
function exact_time_evolution(ansatz,training_thetas; noisy_circuit = false, record = false)
    if record == false
        exact_expvals = Vector{Float64}()
    else
        exact_expvals = Vector{Vector{Float64}}()
    end
    for thetas in training_thetas
        exact_exp_val = exact_trotter_time_evolution(ansatz;special_theta = thetas, noisy_circuit = noisy_circuit, record = record)
        push!(exact_expvals, exact_exp_val)
    end
    return exact_expvals
end

exact_time_evolution (generic function with 1 method)

In [9]:
function final_noise_layer_circuit(ansatz; depol_strength = 0.05, dephase_strength = 0.05)
        #to be replaced with a decent noise model
        depol_noise_layer = [DepolarizingNoise(qind, depol_strength ) for qind in 1:ansatz.nqubits];
        dephase_noise_layer = [DephasingNoise(qind, dephase_strength) for qind in 1:ansatz.nqubits];
        noisy_circuit = deepcopy(ansatz.target_circuit)
        append!(noisy_circuit,depol_noise_layer)
        append!(noisy_circuit,dephase_noise_layer)

        return noisy_circuit
end

final_noise_layer_circuit (generic function with 1 method)

In [10]:
function gate_noise_circuit(ansatz;depol_strength = 0.01, dephase_strength = 0.01, topology=nothing, start_with_ZZ=true, layer = false)
    circuit::Vector{Gate} = []

    if layer == false
        steps = ansatz.steps
    else
        steps = 1
    end

    if isnothing(topology)
        topology = bricklayertopology(ansatz.nqubits)
    end

    # the function after this expects a circuit with at least one layer and will always append something
    if steps == 0
        return circuit
    end

    depol_noise_layer = [DepolarizingNoise(qind, depol_strength ) for qind in 1:ansatz.nqubits];
    phase_damp_layer = [DephasingNoise(qind, dephase_strength) for qind in 1:ansatz.nqubits];

    if start_with_ZZ
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    for _ in 1:steps-1
        rxlayer!(circuit, ansatz.nqubits)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    rxlayer!(circuit, ansatz.nqubits)
    append!(circuit,depol_noise_layer)
    append!(circuit,phase_damp_layer)

    if !start_with_ZZ
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    return circuit
end

gate_noise_circuit (generic function with 1 method)

In [11]:
function cdr(noisy_exp_values::Vector{Float64}, exact_exp_values::Vector{Float64}, noisy_target_exp_value::Float64, exact_target_exp_value::Float64; verbose=false)
    training_data = DataFrame(x=noisy_exp_values,y=exact_exp_values)
    ols = lm(@formula(y ~ x), training_data)
    function cdr_em(x)
        return  coef(ols)[1] + coef(ols)[2] * x
    end
    rel_error_after = abs(exact_target_exp_value - cdr_em(noisy_target_exp_value)) / abs(exact_target_exp_value)
    rel_error_before = abs(exact_target_exp_value - noisy_target_exp_value) / abs(exact_target_exp_value)
    if verbose
        println(training_data)
        println("Noisy target expectation value: ", noisy_target_exp_value)
        println("Relative error before CDR: ", rel_error_before)
        println("CDR-EM target expectation value: ", cdr_em(noisy_target_exp_value))
        println("Relative error after CDR: ", rel_error_after)
    end
    return cdr_em(noisy_target_exp_value), rel_error_after, rel_error_before
end 

cdr (generic function with 1 method)

## MWE
### Exact evolution of a small trotterized circuit (see CPDR p.7)

In [12]:
nq = 4
heaxy_hex_2_topology = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), 
(10, 11), (11, 12), (12, 1), (5, 13), (13, 14), (14, 15), (15, 16), 
(16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 7)]
steps = 20
T = 1.0
J = -5.0 # J > 0 in ferromagnetic phase, J < 0 in antiferromagnetic phase
h = 1.0 #abs(h) < abs(J) in ordered phase
trotter = trotter_setup(nq, steps, T, J, h);
noisy_circuit = gate_noise_circuit(trotter, depol_strength = 0.01, dephase_strength = 0.01);
noisy_circuit_layer = gate_noise_circuit(trotter, depol_strength = 0.01, dephase_strength = 0.01, layer = true);

### Target data

In [13]:
exact_expval_target = exact_trotter_time_evolution(trotter, record = true) #should be close to one as we stay in FM phase

20-element Vector{Float64}:
 0.9950041652780258
 0.9829560484547277
 0.9702825313249236
 0.9619469722102519
 0.9584329190313695
 0.957087782570214
 0.9559873609699532
 0.9562470753200086
 0.960562826272945
 0.9696769062697801
 0.9804909022536117
 0.9878083407048338
 0.9881015452863708
 0.9818659048034192
 0.9726843605892368
 0.9642670717202827
 0.9582829322246882
 0.9545048204736291
 0.9523470614953738
 0.9518557895834706

In [14]:
noisy_expval_target = exact_trotter_time_evolution(trotter; noisy_circuit = noisy_circuit_layer,record = true)

20-element Vector{Float64}:
 0.9752035823889931
 0.9443168763851638
 0.9134495751971731
 0.8866885867684499
 0.864103718403361
 0.8434725688874809
 0.8233598700808222
 0.8044489755148089
 0.7882263316074105
 0.7747692230026499
 0.7619822110550438
 0.7469533949422559
 0.728053238720715
 0.7059017173392615
 0.6825769642661015
 0.6600148376366377
 0.6390043613830174
 0.619352032582033
 0.600677008661402
 0.5829390224783978

In [15]:
plot(0:trotter.steps, exact_expval_target, ylabel="⟨Z⟩", xlabel="time", label="Noise-free", marker=:o)
plot!(0:trotter.steps, noisy_expval_target, label="Noisy", marker=:o, c=2)

### Training data

In [16]:
list_training_thetas = training_set_generation(trotter; num_samples=10, non_cliffs=30)

10-element Vector{Vector{Float64}}:
 [0.7853981633974483, 0.7853981633974483, 0.5, 0.0, 0.0, 0.1, 0.0, 0.7853981633974483, 0.7853981633974483, 0.7853981633974483  …  0.0, 0.0, 0.0, 0.7853981633974483, 0.7853981633974483, 0.7853981633974483, 0.0, 0.0, 0.0, 0.0]
 [0.7853981633974483, 0.5, 0.7853981633974483, 0.0, 0.1, 0.0, 0.0, 0.7853981633974483, 0.7853981633974483, 0.7853981633974483  …  0.0, 0.0, 0.0, 0.5, 0.7853981633974483, 0.7853981633974483, 0.0, 0.0, 0.1, 0.0]
 [0.7853981633974483, 0.7853981633974483, 0.5, 0.0, 0.0, 0.0, 0.0, 0.7853981633974483, 0.7853981633974483, 0.5  …  0.0, 0.0, 0.0, 0.7853981633974483, 0.5, 0.7853981633974483, 0.0, 0.1, 0.0, 0.0]
 [0.7853981633974483, 0.7853981633974483, 0.7853981633974483, 0.0, 0.0, 0.1, 0.1, 0.7853981633974483, 0.5, 0.7853981633974483  …  0.1, 0.0, 0.0, 0.7853981633974483, 0.5, 0.7853981633974483, 0.1, 0.0, 0.0, 0.0]
 [0.7853981633974483, 0.7853981633974483, 0.5, 0.0, 0.0, 0.1, 0.1, 0.7853981633974483, 0.5, 0.7853981633974483  …  0.0, 0.0,

In [17]:
exact_expvals = exact_time_evolution(trotter,list_training_thetas)

10-element Vector{Float64}:
 0.9824399370754818
 0.9741040831600327
 0.9731751176349954
 0.9447809154200373
 0.9755896635916715
 0.9798127313927876
 0.9812773985928911
 0.9958605839146614
 0.9872335701535768
 0.9952384754534502

In [18]:
noisy_expvals = exact_time_evolution(trotter,list_training_thetas; noisy_circuit = noisy_circuit)

10-element Vector{Float64}:
 0.6529797585455857
 0.6498460852139263
 0.6491223687101924
 0.6369099382684054
 0.6506928448118635
 0.6528545686612767
 0.6544056428254187
 0.6627935642708517
 0.6587825136673917
 0.6580009630269585

In [19]:
noisy_circ = gate_noise_circuit(trotter, depol_strength = 0.01, dephase_strength = 0.01);

In [20]:
corr_energy, rel_error_after, rel_error_before = cdr(noisy_expvals, exact_expvals, noisy_expval_target[end], exact_expval_target[end]; verbose=true)  

10×2 DataFrame
 Row │ x         y        
     │ Float64   Float64  
─────┼────────────────────
   1 │ 0.65298   0.98244
   2 │ 0.649846  0.974104
   3 │ 0.649122  0.973175
   4 │ 0.63691   0.944781
   5 │ 0.650693  0.97559
   6 │ 0.652855  0.979813
   7 │ 0.654406  0.981277
   8 │ 0.662794  0.995861
   9 │ 0.658783  0.987234
  10 │ 0.658001  0.995238
Noisy target expectation value: 0.5829390224783978
Relative error before CDR: 0.38757632315974017
CDR-EM target expectation value: 0.8388041515090026
Relative error after CDR: 0.11876971208415839


(0.8388041515090026, 0.11876971208415839, 0.38757632315974017)

## Next steps:
- DONE: make realistic noise model (the other naive noise model doesnt work with record/layers!!)
- change observable to where correction is not as easy
- other breaking points: resolution of the timesteps, i.e. smaller gate angels (wont work with Clifford replacement) 